# Using evolutionary algorithms to optimize graphql requests

**The brief overview of the idea**

TODO: describe

*Step 1. Backend server in fast api with graphql endpoint*

After start, goto: http://127.0.0.1:8000/graphql

Request example:
```
{
  allPeople {
    id
    name
    age
  }
}
```

In [1]:
import json
import strawberry
from fastapi import FastAPI
from strawberry.fastapi import GraphQLRouter
import uvicorn
import nest_asyncio

# Needed to allow uvicorn to run inside Jupyter
nest_asyncio.apply()

# Load your JSON data
with open("../resources/sets/data-1.json") as f:
    people_data = json.load(f)

@strawberry.type
class Person:
    id: int
    name: str
    age: int

@strawberry.type
class Query:
    @strawberry.field
    def all_people(self) -> list[Person]:
        return [Person(**person) for person in people_data]

# Setup GraphQL
schema = strawberry.Schema(query=Query)
graphql_app = GraphQLRouter(schema)

# Create FastAPI app
app = FastAPI()
app.include_router(graphql_app, prefix="/graphql")

# Run the server inline
uvicorn.run(app, host="127.0.0.1", port=8000)

# ../resources/sets/data-1.json

INFO:     Started server process [39176]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [39176]


Run in helpers folder:

```
python server.py
```

Then execute the following code: 

In [ ]:
import requests
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

# GraphQL endpoint
url = "http://127.0.0.1:8000/graphql"

# GraphQL query
query = """
{
  allPeople {
    id
    name
    age
  }
}
"""

queryLocations = """
{
  locations {
    locationId
    locationName
    hasChildren {
      locationId
      hasChildren {
        locationName
      }
    }
    visits {
      purpose
      person {
        name
        email
      }
    }
  }
}
"""

def make_request(request_query):
    response = requests.post(url, json={"query": request_query})
    
    if response.status_code == 200:
        data = response.json()
        print(f"Success: {len(data['data']['locations'])} locations retrieved")
    else:
        print(f"Failed with status code {response.status_code}")

total_requests = 100
num_threads = 10

# Start timing
start_time = time.time()

# Loop 100 times
# Run in threads
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(make_request, queryLocations) for i in range(total_requests)]
    for future in as_completed(futures):
        pass
    
# Not in threads: slower (a bit)
# for i in range(100):
#     make_request(query, i)

# End timing
end_time = time.time()

# Total duration
duration = end_time - start_time
print(f"\nTotal time for 100 requests: {duration:.2f} seconds")

Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people retrieved
Success: 4 people re